<a href="https://colab.research.google.com/github/Lenin1999/WebScraping-Dispositivos-Moviles/blob/main/webScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalación de las librerias o dependencias necesarias
!pip install beautifulsoup4 requests pandas

In [ ]:
# Load dependencies
import json # modulo para trabajar con archivos JSON
import requests # libreria para hacer peticiones HTTP
import pandas as pd # libreria para procesar datos, proporciona funciones para analíticas
from bs4 import BeautifulSoup as bs # Extraer información de paginas web

In [ ]:
# Función que convierte una lista de elementos en un dataframe de pandas
def rowsToDataFrame(rows):
    df = pd.DataFrame(rows[1:], columns=rows[0])
    return df

In [ ]:
def processTableData(tbl):
    rows = []  # Lista para almacenar las filas procesadas
    for child in tbl.find('tbody').children:
        if child.name == 'tr':  # Asegurarse de que solo estemos trabajando con las filas
            row = []  # Lista para almacenar los elementos de una fila
            first_column = True  # Variable para indicar si estamos procesando la primera columna (nombres de los equipos)
            for td in child.find_all(['td', 'th']):
                try:
                    item = td.text.replace('\n', '')  # Obtener el texto del elemento, eliminando saltos de línea
                    if item:
                        if first_column:
                            row.append(item)  # Agregar el elemento directamente si es la primera columna(Para no perder los nombres del pais)
                            first_column = False  # Cambiar la variable después de agregar el primer elemento
                        else:
                            item = ''.join(filter(str.isdigit, item))  # Eliminar caracteres no numéricos y unir los dígitos
                            row.append(int(item))  # Convertir a número entero y agregar a la fila
                    else:
                        row.append(0)  # Si el elemento está vacío, agregar 0 a la fila
                except:
                    continue  # Ignorar cualquier excepción y continuar con la siguiente iteración
            rows.append(row)  # Agregar la fila completa a la lista de filas
    return rows  # Devolver la lista de filas procesadas





In [ ]:
# Función para extraer la data de la URL y procesar el contenido HTML
def processDataHTML(data):
    soup = bs(data['sections'][1]['text'], 'html.parser') # Recupera el item del JSON y lo convierte en contenido HTML
    tbl = soup.find_all('table')[0] # Recupera la tabla del contenido HTML
    # print(tbl.prettify())
    tblRows = processTableData(tbl) # Ejecuta la función apra porcesar el contenido de la Tabla
    return tblRows

In [ ]:
# Read website
# r = requests.get('https://www.wikiwand.com/es/Anexo:Tabla_estadística_de_la_Copa_Mundial_de_Fútbol')
r = requests.get('https://www.wikiwand.com/mcs-api/en.wikipedia.org/v1/page/mobile-sections-remaining/Motocross_des_Nations')  # Recuperamos el contenido de la web
# print(r)
table = processDataHTML(json.loads(r.content)) # Formate el contenido en formato JSON
df = rowsToDataFrame(table) # ejecuta la funcion para recupera los datos de la tabla como dataframe de Pandas
df

,Team,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2021,2022,2023
0,Argentina,0,0,0,0,0,0,0,0,0,...,0,0,0,0,30,22,0,0,0,0.0
1,Australia,7,6,0,14,13,16,11,20,6,...,4,14,7,8,6,4,15,0,3,2.0
2,Austria,13,0,0,0,22,23,21,0,0,...,12,0,10,15,0,12,17,9,0,0.0
3,Belarus,0,0,0,27,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0.0
4,Belgium,3,2,2,1,1,3,2,3,3,...,1,2,3,4,4,6,2,6,5,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,Thailand,0,0,0,0,0,0,0,0,0,...,39,0,0,0,36,0,0,0,0,0.0
58,United States,1,0,0,2,0,1,1,1,1,...,2,3,2,3,9,5,6,0,1,8.0
59,Ukraine,31,28,0,0,0,0,0,21,0,...,29,30,31,33,28,26,23,28,0,36.0
60,Venezuela,26,27,15,0,0,0,0,24,31,...,33,0,29,0,0,14,0,23,20,0.0


In [ ]:
df.to_csv('DatosMotocross2000-2023.csv')

In [ ]:
año = 2022

# Encontrar los equipos que han tenido posiciones de 1, 2 o 3 en el año definido
equipos_posiciones_123 = df[(df[año] == 1) | (df[año] == 2) | (df[año] == 3)]

# Imprimir los equipos y sus posiciones en el año definido
print(f"Equipos con posiciones de 1, 2 o 3 en {año}:")
print(equipos_posiciones_123[['Team', año]])



Equipos con posiciones de 1, 2 o 3 en 2022:
              Team  2022
1        Australia     3
21          France     2
32           Italy     3
58   United States     1
